In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Define the stress calculation function
def strain_rate_dependent_stress(sr, C1, C2, C3, C4):
    return C1 * np.log(sr) + C2 * sr**C3 + C4 * (np.log(sr))**2

# Load data (replace with your file paths)
file_1 = pd.read_csv('Fitted_Parameters_Tmp298.15_RD.csv')
file_2 = pd.read_csv('Parameters_Tmp298.15_RD_new.csv')

# Strain rate range for predictions
strain_rates = np.logspace(-4, 0, 100)  # From 0.0001 to 1 (logarithmic scale)

# Calculate stresses for each strain
stress_comparison = []
for strain in file_1['strain']:
    params_1 = file_1[file_1['strain'] == strain].iloc[0]
    params_2 = file_2[file_2['PEEQ'] == strain].iloc[0]

    stresses_1 = [strain_rate_dependent_stress(sr, params_1['C1'], params_1['C2'], params_2['C3'], params_2['C4']) for sr in strain_rates]
    stresses_2 = [strain_rate_dependent_stress(sr, params_2['C1'], params_2['C2'], params_2['C3'], params_2['C4']) for sr in strain_rates]

    stress_comparison.append({
        'strain': strain,
        'strain_rates': strain_rates,
        'stresses_file_1': stresses_1,
        'stresses_file_2': stresses_2
    })

# Create a DataFrame for comparison
comparison_df = pd.DataFrame(stress_comparison)

# Save the plots to a PDF file
optimized_pdf_path = 'Stress_Comparison_Optimized_Plots.pdf'
pdf_pages = PdfPages(optimized_pdf_path)

# Plot stress comparisons and save them to the PDF
for _, row in comparison_df.iterrows():
    plt.figure()
    plt.plot(row['strain_rates'], row['stresses_file_1'], label='File 1', marker='o', linestyle='-')
    plt.plot(row['strain_rates'], row['stresses_file_2'], label='File 2', marker='x', linestyle='--')
    plt.xscale('log')
    plt.xlabel('Strain Rate')
    plt.ylabel('Stress')
    plt.title(f"Stress Comparison for Strain = {row['strain']:.5f}")
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    pdf_pages.savefig()
    plt.close()

pdf_pages.close()

print(f"PDF saved to {optimized_pdf_path}")


PDF saved to Stress_Comparison_Optimized_Plots.pdf
